# Exponential Map

In [ ]:
import numpy as np

from sympy.algebras.quaternion import Quaternion

from scipy.linalg import expm, logm

In [ ]:
def C(x,y):
    return x@y - y@x

def Q(x,y):
    return x*y - y*x

def R(x,y):
    return np.cross(x,y)-np.cross(y,x)

def baker(x,y,B):
    return x+y + B(x,y)/2 + B(x,B(x,y))/12 + B(y,B(x,y))/12

deg = np.pi/180

Ejemplo simple en Aff(1):

In [ ]:
m1 = np.array( [[1,2],[0,1]] )
m2 = np.array( [[4,3],[0,1]] )
t1 = logm(m1)
t2 = logm(m2)

In [ ]:
m1@m2

In [ ]:
expm(t1+t2)

In [ ]:
expm(baker(t1,t2,C))

In [ ]:
m2@m1

In [ ]:
expm(baker(t2,t1,C))

Ejemplo en SO(3)

In [ ]:
def rot3(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[c, -s, 0]
                  ,[s,  c, 0]
                  ,[0,  0, 1]])

def rot1(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[1, 0,  0]
                  ,[0, c, -s]
                  ,[0, s,  c]])

def rot2(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[ c, 0, s]
                  ,[ 0, 1, 0]
                  ,[-s, 0, c]])

In [ ]:
m1 = rot3(30*deg)
m2 = rot1(40*deg)
t1 = logm(m1)
t2 = logm(m2)

In [ ]:
m1 @ m2

In [ ]:
expm(t1+t2)

In [ ]:
expm(baker(t1,t2,C))

In [ ]:
m2 @ m1

In [ ]:
expm(baker(t2,t1,C))

In [ ]:
t1

In [ ]:
u1 = np.array([0,0,1])
a1 = 30*deg
u2 = np.array([1,0,0])
a2 = 40*deg

q1 = Quaternion(np.cos(a1/2),*(np.sin(a1/2)*u1))
q2 = Quaternion(np.cos(a2/2),*(np.sin(a2/2)*u2))

In [ ]:
q1

In [ ]:
q1.to_axis_angle()

In [ ]:
q1.to_rotation_matrix()

In [ ]:
m1

In [ ]:
q2

In [ ]:
q2.to_axis_angle()

In [ ]:
q2.to_rotation_matrix()

In [ ]:
m2

In [ ]:
m2@m1

In [ ]:
np.array([(q2*q1*Quaternion(0,*b)*q1.inverse()*q2.inverse()).args[1:] for b in np.eye(3)]).T

In [ ]:
tq1=Quaternion(0,0,0,a1/2)
tq2=Quaternion(0,a2/2,0,0)

In [ ]:
tq1.exp()

In [ ]:
baker(tq2,tq1,Q)

In [ ]:
baker(tq2,tq1,Q).exp()

In [ ]:
baker(tq2,tq1,Q).exp().to_rotation_matrix()

In [ ]:
v1 = np.array(tq1.args[1:]).astype(float)
v2 = np.array(tq2.args[1:]).astype(float)

In [ ]:
baker(v2,v1,R)